In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data=pd.read_csv('/content/drive/My Drive/Sentiment.csv')

In [0]:
data=data[['text','sentiment']]

In [55]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [0]:
trr=[]
srr=[]
t=0
for i in data['sentiment']:
  if(i!="Neutral"):
    trr.append(data.iloc[t][0])
    srr.append(data.iloc[t][1])
  t+=1

In [0]:
from keras.models import Model
from keras.layers import Input

In [0]:
trr=[i.lower() for i in trr]
trr=[re.sub('[^a-zA-z0-9\s]','',x) for x in trr]
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(trr)
X = tokenizer.texts_to_sequences(trr)
X = pad_sequences(X)
Y=pd.get_dummies(srr).values

In [59]:
X[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  17, 282,
       250,   6, 811, 103, 171,  27, 135,   7,   1, 172,  13,   2, 232,
       715,  18], dtype=int32)

In [60]:
trr[1]

'rt robgeorge that carly fiorina is trending  hours after her debate  above any of the men in justcompleted gopdebate says shes on '

In [61]:
Y.shape

(10729, 2)

In [62]:
max_features=2000
embedded_dim=128
lstm_out=156
input_shape=(28,)
sentence_indices=Input(input_shape,dtype='int32')
embedding_layer=Embedding(max_features,embedded_dim)(sentence_indices)
lstm=LSTM(lstm_out,dropout=0.5)(embedding_layer)
y=Dense(2,activation='relu')(lstm)
model=Model(inputs=sentence_indices,outputs=y)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 28, 128)           256000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 156)               177840    
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 314       
Total params: 434,154
Trainable params: 434,154
Non-trainable params: 0
_________________________________________________________________


In [63]:
srr

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',

In [64]:
from sklearn.preprocessing import LabelEncoder
m1=LabelEncoder()
srr1=m1.fit_transform(srr)
srr1

array([1, 1, 1, ..., 1, 0, 1])

In [65]:
Y=pd.get_dummies(srr1).values
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
y_train

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [66]:
model.fit(x_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
8583/8583 [==============================] - 22s 3ms/step - loss: 3.3308 - acc: 0.7912
Epoch 2/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3389 - acc: 0.7919
Epoch 3/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3446 - acc: 0.7919
Epoch 4/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3464 - acc: 0.7919
Epoch 5/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3333 - acc: 0.7919
Epoch 6/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3427 - acc: 0.7919
Epoch 7/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3521 - acc: 0.7919
Epoch 8/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3521 - acc: 0.7919
Epoch 9/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3408 - acc: 0.7919
Epoch 10/10
8583/8583 [==============================] - 21s 2ms/step - loss: 3.3446 - acc: 0.7919


In [0]:
xrr=model.predict(x_test)

In [71]:
print(model.evaluate(x_test,y_test,batch_size=32))

2146/2146 [==============================] - 2s 910us/step
[3.3723322024216524, 0.7903075489282386]
